# How to use metrics on the surface of a sphere (of Earth Radius)
Most of the analysis were performed using a regular longitude-latitude grid.<br>
Here we show how to work on a sphere.
## Load packages

In [ ]:
using Makie, CairoMakie
using DIVAnd

### Function to plot on a sphere

In [ ]:
function plot_sphere(fi::Matrix{Float64}; cmap = :RdYlBu)
    fig = Figure()
    ax = Axis3(fig[1, 1], aspect = :equal, azimuth = 1.75 * π)
    surface!(
        ax,
        xi,
        yi,
        zi,
        color = (fi .- minimum(fi)) / (maximum(fi) - minimum(fi)),
        colormap = cmap
    )
    scatter!(ax, xobs, yobs, zobs, color = :black)
    hidedecorations!(ax)
    hidespines!(ax) 
    display(fig)
    return fig
end

## Prepare grid and coordinates

In [ ]:
loni, lati =
    ndgrid(range(0.0, stop = 360.0, length = 200), range(-89, stop = 89, length = 210));

# take out last longitude to not close, that will be done by moddim
#
loni = loni[1:end-1, :]
lati = lati[1:end-1, :]

R = 6371.009e3

# Cartesian coordinates for the plotting on a sphere
xi = R * cosd.(loni) .* cosd.(lati);
yi = R * sind.(loni) .* cosd.(lati);
zi = R * sind.(lati);

mask = trues(size(loni));

## Naive analysis as if coordinates were cartesian
We consider 3 observations.

In [ ]:
lon = [2, 300, 270]
lat = [45, 0.0, 80]
f = [1.0, -1.0, -1.0]

pmc = ones(size(loni)) / (loni[2, 1] - loni[1, 1]);
pnc = ones(size(lati)) / (lati[1, 2] - lati[1, 1]);
fi, s = DIVAndrun(mask, (pmc, pnc), (loni, lati), (lon, lat), f, (10.0, 10.0), 1.0);

Observation coordinates (for plotting only)

In [ ]:
xobs = R * cosd.(lon) .* cosd.(lat);
yobs = R * sind.(lon) .* cosd.(lat);
zobs = R * sind.(lat);

### Plot results
Look at the field near the observation located close to the pole.

In [ ]:
f1 = plot_sphere(fi)
save("sphere.png", f1)

## With correct metrics and correlation length in the same metric (meters)
<div class="alert alert-block alert-info">
ℹ️ Note that the metric function works with Earth Radius. <br>
If you have a sphere of a different radius, just scale the <code>pm</code> accordingly (<code>pm</code> are inverse of length).
</div>

In [ ]:
pm, pn = DIVAnd_metric(loni, lati)
fi, s = DIVAndrun(mask, (pm, pn), (loni, lati), (lon, lat), f, (1000000.0, 1000000.0), 1.0);

### Plot results
The field is improved near the North pole, but the discontinuity at 180° has to be addressed.

In [ ]:
plot_sphere(fi)
save

## Adding periodicity in the analysis 
(not in the plotting mesh). This is done with the argument <code>moddim</code>.

In [ ]:
pm, pn = DIVAnd_metric(loni, lati)
lon = [2, 300, 270]
lat = [45, 0.0, 80]
f = [1.0, -1.0, -1.0]
fi, s = DIVAndrun(
    mask,
    (pm, pn),
    (loni, lati),
    (lon, lat),
    f,
    (1000000.0, 1000000.0),
    1.0,
    moddim = [1, 0],
);

### Plot results
The continuity is now assured.

In [ ]:
plot_sphere(fi)